In [1]:
#libraries 
import os
import pandas as pd
import numpy as np

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#accuracy measures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

import pickle


In [2]:
#path = '/Users/mustakimsunny/Desktop/ML/hasin_SGI_project/sgi_project/data/'
#path = '/Users/tasrifahmed'
#file = 'NCDB_1999_to_2017.csv'
dataset = pd.read_csv("/Users/tasrifahmed/PyProjects/untitled/NCDB_1999_to_2017.csv")

/Users/tasrifahmed/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Cleaning form unwanted data
df = dataset[(dataset.C_MNTH != 'UU') & (dataset.C_WDAY != 'U') & (dataset.C_HOUR != 'UU') & (dataset.C_VEHS != 'UU') &
            (dataset.C_CONF != 'QQ') & (dataset.C_CONF != 'UU') & (dataset.C_RCFG != 'UU') & (dataset.C_RCFG != 'QQ') &
            (dataset.C_WTHR != 'U') & (dataset.C_WTHR != 'Q') & (dataset.C_RSUR != 'Q') & (dataset.C_RSUR != 'U') & 
            (dataset.C_RALN != 'U') & (dataset.C_RALN != 'Q') & (dataset.C_TRAF != 'QQ') & (dataset.C_TRAF != 'UU') &
            (dataset.V_ID != 'UU') & (dataset.V_TYPE != 'UU') & (dataset.V_TYPE != 'NN') & (dataset.V_TYPE != 'QQ') & 
            (dataset.V_YEAR != 'NNNN') & (dataset.V_YEAR != 'UUUU') & (dataset.P_ID != 'NN') & (dataset.P_ID != 'UU') &
            (dataset.P_SEX != 'U') & (dataset.P_SEX != 'N') & (dataset.P_AGE != 'UU') & (dataset.P_AGE != 'NN') & 
            (dataset.P_PSN != 'QQ') & (dataset.P_PSN != 'UU') & (dataset.P_PSN != 'NN') & (dataset.P_ISEV != 'N') & 
             (dataset.P_ISEV != 'U') & (dataset.P_SAFE != 'QQ') & (dataset.P_SAFE != 'NN') & (dataset.P_SAFE != 'UU') & 
            (dataset.P_USER != 'U')]

In [4]:
# Make all coumns int

def make_int(df, *arg):
    for col in arg:
        df[col] = df[col].astype(int)
    return df

df = make_int(df, 'C_YEAR','C_MNTH','C_WDAY','C_HOUR','C_SEV','C_VEHS','C_CONF','C_RCFG','C_WTHR','C_RSUR',
              'C_RALN','C_TRAF','V_ID','V_TYPE','V_YEAR','P_ID','P_AGE','P_PSN','P_ISEV','P_SAFE','P_USER','C_CASE')

<ipython-input-4-050eb65e980d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(int)


In [5]:
#make all column categorical data
def convert_to_cat(df, *arg):
    for column in arg:
        df[column] = df[column].astype('category')
        df[column] = df[column].cat.codes
        
    return df

df = convert_to_cat(df, 'C_YEAR','C_MNTH','C_WDAY','C_HOUR','C_SEV','C_VEHS','C_CONF','C_RCFG','C_WTHR','C_RSUR',
              'C_RALN','C_TRAF','V_ID','V_TYPE','V_YEAR','P_ID','P_AGE','P_PSN','P_ISEV','P_SAFE','P_USER','C_CASE', 'P_SEX')

<ipython-input-5-8e146d2ae008>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype('category')
<ipython-input-5-8e146d2ae008>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].cat.codes


In [6]:
#seperate target col from other features to see feature importance
X = df.drop(['C_RCFG', 'C_CASE'], axis=1)  #independent columns
y = df['C_RCFG']    #target column i.e price range

In [7]:
#univariate selection

import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=20)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

     Specs         Score
10  C_TRAF  1.312417e+07
6   C_CONF  2.095772e+06
9   C_RALN  3.034329e+05
8   C_RSUR  1.476200e+05
16   P_AGE  7.868552e+04
7   C_WTHR  7.836030e+04
0   C_YEAR  4.335337e+04
12  V_TYPE  2.944987e+04
18  P_ISEV  1.799216e+04
3   C_HOUR  1.754756e+04
5   C_VEHS  1.273484e+04
11    V_ID  6.787337e+03
13  V_YEAR  5.482935e+03
2   C_WDAY  3.242556e+03
14    P_ID  2.444056e+03
15   P_SEX  1.924622e+03
20  P_USER  1.599623e+03
17   P_PSN  8.623420e+02
19  P_SAFE  3.249773e+02
1   C_MNTH  2.722133e+02


In [8]:
#drop cols that are not important after feature engineering
def drop_cols(df,*arg):
    columns = list(arg)
    df = df.drop(columns, axis=1)
    return df

df = drop_cols(df,'V_ID','P_PSN','C_CASE','P_SAFE','C_RSUR','C_RALN','P_USER','P_SEX','V_TYPE','C_SEV','P_ISEV','P_ID')

In [9]:
#looks like the class values are heavily imbalanced
df['C_RCFG'].value_counts()

1    2014779
0    1523700
2     218917
4      35001
3      11838
7       7185
5       4279
8       1299
6        364
9        251
Name: C_RCFG, dtype: int64

In [10]:
#upsample data to balance
def balance_dataset(dataset, column, frequency_val):
    count = dataset[column].value_counts()
    classes = [key for key,val in count.items()]
    df_majority = dataset.loc[dataset[column] == classes[0]]
    
    all_minor_dfs = []
    for val in classes[1:]:
        all_minor_dfs.append(dataset.loc[dataset[column] == val])
    
    # Upsample minority class
    all_minor_dfs_upsampled = []
    for val in all_minor_dfs:
        df_minority_upsampled = resample(val, 
                                     replace=True,     # sample with replacement
                                     n_samples=frequency_val,    # to match majority class
                                     random_state=123) # reproducible results
        all_minor_dfs_upsampled.append(df_minority_upsampled)
        
    #concatenate majority class with matching upsampled classes
    #for df in all_minor_dfs_upsampled:
    df_balanced = pd.concat([df_majority, all_minor_dfs_upsampled[0]])
    for df in all_minor_dfs_upsampled[1:]:
        df_balanced = pd.concat([df_balanced, df])
    
    df_balanced = shuffle(df_balanced)
    return df_balanced

df_balanced = balance_dataset(df, 'C_RCFG',2014779)

In [14]:
df_balanced['C_RCFG'].value_counts()

9    2014779
8    2014779
7    2014779
6    2014779
5    2014779
4    2014779
3    2014779
2    2014779
1    2014779
0    2014779
Name: C_RCFG, dtype: int64

In [11]:
df_balanced = df_balanced[:40000]

In [12]:
df_balanced

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_TRAF,V_YEAR,P_AGE
575106,1,5,0,8,4,6,4,2,16,73,31
4098105,10,6,3,17,1,6,3,1,16,88,38
458137,1,1,2,18,1,13,3,3,16,91,34
1082244,2,7,3,20,0,5,6,0,16,86,31
141071,0,4,4,13,3,6,4,2,16,78,23
...,...,...,...,...,...,...,...,...,...,...,...
4631445,12,1,1,13,2,15,2,0,0,100,79
2914091,7,2,2,8,1,6,3,1,16,93,50
905064,2,2,1,12,1,6,7,2,3,77,19
262192,0,7,5,21,1,14,7,1,16,89,42


In [13]:
#split into train and test
def split_train_test(dataframe, target):
    train_set, test_set = train_test_split(dataframe, test_size=0.2, random_state=42)
    X_train, X_test = train_set.drop(target, axis=1), test_set.drop(target, axis=1)
    y_train, y_test = train_set[target], test_set[target]
    
    return X_train, X_test, y_train, y_test 

X_train, X_test, y_train, y_test = split_train_test(df_balanced, 'C_RCFG')

In [14]:
#algorithms = [KNeighborsClassifier(), DecisionTreeClassifier(),RandomForestClassifier(), GaussianNB()]
def train_model(X_train, y_train):
    model = RandomForestClassifier()   # DecisionTreeClassifier() and  KNeighborsClassifier() and GaussianNB()
    model.fit(X_train, y_train)
    
    return model
model = train_model(X_train, y_train)

In [18]:
def test_model (X_train, X_test, y_test):
    y_test_pred = model.predict(X_test)
    y_tain_pred = model.predict(X_train)
    
    return  y_tain_pred,y_test_pred

y_train_pred, y_test_pred = test_model (X_train, X_test, y_test)

In [19]:
#model stat
def model_accuracy_stat(y_test, y_test_pred, y_train, y_train_pred):
    print('test accuracy: ', accuracy_score(y_test,y_test_pred))
    print('train accuracy: ', accuracy_score(y_train, y_train_pred))
    #print(classification_report(y_test, y_test_pred))
    print('confusion matrix: \n', confusion_matrix(y_test, y_test_pred))

model_accuracy_stat(y_test, y_test_pred, y_train, y_train_pred)

test accuracy:  0.725875
train accuracy:  1.0
confusion matrix: 
 [[266  32  78  64 205  79   3  71  18   8]
 [ 30 589  48  41  39   8   3  25  12   1]
 [100  83 341  98  83  29   1  15  11   1]
 [ 44  46  48 541  67  12   2  20   8   3]
 [134  50  37  26 447  56   1  48  15   0]
 [ 45   8  12  19  46 622   2  15   3   1]
 [  0   0   0   0   0   0 815   0   0   0]
 [ 46  39  18  22  48  18   2 624  17   2]
 [  8   4   0   2   5   2   0   6 759   0]
 [  0   0   0   0   0   0   0   0   0 803]]
